In [4]:
import polars as pl
import json

Create 2 datasets:

1. Definitions that ARE in the corpus
    This is used to evaluate the retrieval. Possibly divided by language?

2. Definitions that ARE NOT in the corpus
    This is used to evaluate the generation.


Evaluation metrics:
- BLEU
This is used to measure lexical similarity
- BertScore
This is used to measure semantic similarity

Possibly also:
- BLEURT
This is used to measure the quality of the generated definitions. Or maybe just use 

---
## Evaluation dataset preparation

### LEOS

In [5]:
leos_eval_dataset = pl.read_excel("../data/Evaluation_LEOS_annotato_def.xlsx")
leos_eval_dataset

CELEX,Act,EuroVoc,Selected_EuroVoc,Def_no,Definition,Suggestion_1,Suggestion_2,Suggestion_3,Suggestion_4,Suggestion_5,Relevance_1,Relevance_2,Relevance_3,Relevance_4,Relevance_5
str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str
"""32016R0679""","""GDPR""","""natural person; protection of…","""protection of privacy; data…",1,"""Non-personal Data""","""data means data other than pe…","""identity data means the data …","""biometric data means fingerpr…","""identity data means the data …","""biometric data means fingerpr…","""TP""","""TP""","""TP""","""TP""","""TP"""
"""32016R0680""","""GDPR""","""natural person; protection of…","""protection of privacy; data-p…",2,"""Connected device""","""nomadic device means a portab…","""investigational device means …","""device means active implantab…","""connected infrastructure mean…","""connected undertakings means:…","""TP""","""FP""","""TP""","""FP""","""FP"""
"""32016R0681""","""GDPR""","""natural person; protection of…","""protection of privacy; data…",3,"""IoT device""","""nomadic device means a portab…","""investigational device means …","""device means active implantab…","""qualified electronic signature…","""qualified electronic seal crea…","""TP""","""FP""","""TP""","""FP""","""FP"""
"""32016R0682""","""GDPR""","""natural person; protection of…","""protection of privacy; data…",4,"""Open Data""","""open format means open format…","""data means data other than pe…","""open category' means a catego…","""biometric data means personal…","""biometric data means personal…","""TP""","""TP""","""FP""","""FP""","""FP"""
"""32016R0683""","""GDPR""","""natural person; protection of…","""protection of privacy; data…",5,"""Satellite Data""","""data means data other than pe…","""biometric data means personal…","""biometric data means personal…","""biometric data means personal…","""identity data means the data …","""TP""","""FP""","""FP""","""FP""","""FP"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""32016L0685""","""Law Enforcement Directive""","""fight against crime; protecti…","""fight against crime; protecti…",6,"""Traffic Data""","""traffic data means historic a…","""data traffic records means re…","""biometric data means personal…","""road data means data on road …","""genetic data means personal d…","""FP""","""FP""","""FP""","""FP""","""FP"""
"""32016L0686""","""Law Enforcement Directive""","""fight against crime; protecti…","""fight against crime; protecti…",7,"""GPS Data""","""biometric data means personal…","""genetic data means personal d…","""data means data other than pe…","""personal data breach means a …","""data concerning health means …","""FP""","""FP""","""TP""","""FP""","""FP"""
"""32016L0687""","""Law Enforcement Directive""","""fight against crime; protecti…","""fight against crime; protecti…",8,"""International Cooperation Poli…","""international organisation me…","""international organisation me…","""international organisation me…","""international organisation me…","""regional cooperation means co…","""FP""","""FP""","""FP""","""FP""","""TP"""


In [12]:
leos_dict = leos_eval_dataset.select(["CELEX", "Definition"]).rename({"CELEX": "CELEX_ID", "Definition": "Term"}).to_dicts()
leos_dict

[{'CELEX_ID': '32016R0679', 'Term': 'Non-personal Data'},
 {'CELEX_ID': '32016R0680', 'Term': 'Connected device'},
 {'CELEX_ID': '32016R0681', 'Term': 'IoT device'},
 {'CELEX_ID': '32016R0682', 'Term': 'Open Data'},
 {'CELEX_ID': '32016R0683', 'Term': 'Satellite Data'},
 {'CELEX_ID': '32016R0684', 'Term': 'Cookie'},
 {'CELEX_ID': '32016R0685', 'Term': 'Synthetic Data'},
 {'CELEX_ID': '32016R0686', 'Term': 'International Data Transfer'},
 {'CELEX_ID': '32016R0687', 'Term': 'Data Sharing activity'},
 {'CELEX_ID': '32016R0688', 'Term': 'Data Buyer'},
 {'CELEX_ID': '32011L0083', 'Term': 'Connected device'},
 {'CELEX_ID': '32011L0084', 'Term': 'IoT device'},
 {'CELEX_ID': '32011L0085', 'Term': 'Dangerous Product'},
 {'CELEX_ID': '32011L0086', 'Term': 'Dangerous Product Importer'},
 {'CELEX_ID': '32011L0087', 'Term': 'Intermediary Platform'},
 {'CELEX_ID': '32011L0088', 'Term': 'Internet Platform'},
 {'CELEX_ID': '32011L0089', 'Term': 'Content Creator'},
 {'CELEX_ID': '32011L0090', 'Term': '

In [13]:
lexdrafter = [
    {
        "Term": "rules",
        "CELEX_ID": "32021Q0618(02)",
        "original_definition": "'rules' means these Rules of the Supply Agency of the European Atomic Energy Community determining the manner in which demand is to be balanced against the supply of ores, source materials and special fissile materials;",
    },
    {
        "Term": "generic ecodesign requirement",
        "CELEX_ID": "32009L0125",
        "original_definition": "'generic ecodesign requirement' means any ecodesign requirement based on the ecological profile as a whole of a product without set limit values for particular environmental aspects;",
    },
    {
        "Term": "refrigerating appliance",
        "CELEX_ID": "32019R2016",
        "original_definition": "'refrigerating appliance' means an insulated cabinet with one or more compartments that are controlled at specific temperatures, cooled by natural or forced convection whereby the cooling is obtained by one or more energy consuming means;",
    },
    {
        "Term": "abandoned land",
        "CELEX_ID": "32019R0807",
        "original_definition": "'abandoned land' means unused land, which was used in the past for the cultivation of food and feed crops but where the cultivation of food and feed crops was stopped due to biophysical or socioeconomic constraints;",
    },
    {
        "Term": "adequacy",
        "CELEX_ID": "32017R1485",
        "original_definition": "'adequacy' means the ability of in-feeds into an area to meet the load in that area;",
    },
    {
        "Term": "aggregates",
        "CELEX_ID": "32008R1099",
        "original_definition": "'aggregates' means data aggregated at national level on the treatment or use of energy products, namely production, trade, stocks, transformation, consumption, and structural characteristics of the energy system such as installed capacities for electricity generation or production capacities for oil products;",
    }
]


In [1]:
from LegalDefAgent.src.definition_search import query_vectorstore


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [14]:
for term in leos_dict:
    res = query_vectorstore(term['Term'])
    print(term['Term'])
    print([r['metadata']['definendum_label'] for r in res])
    print()


Non-personal Data
['#detailedData', '#aggregatedData', '#aggregatedData', '#personalData', '#non-householdCustomer', '#datiPersonali', '#personalData', '#data', '#datiPersonali', '#restrictedData']

Connected device
['#activeDevice', '#device', '#user', '#user', '#towingDevice', '#batteryCharger', '#nomadicDevice', '#interconnector', '#RefuellingConnection', '#deviceForNear-PatientTesting']

IoT device
['#device', '#MeansADevice', '#medicalDevice', '#oggettoDellaFornitura', '#medicalDevice', '#oggettoDellaFornitura', '#informationTechnologyEquipment', '#investigationalDevice', '#SafetyDevice', '#dispositivoMedico']

Open Data
['#metadata', '#openFormat', '#Scopus', '#log', '#ricercaAutomatizzata', '#varietaPubblica', '#openAccess', '#CopernicusIn-SituData', '#dataOriginator', '#EndOfDataCollection']

Satellite Data
['#vesselMonitoringSystemData', '#SSTData', '#data', '#MeansAMeansByWhichAircraft', '#aerodromeMappingData', '#CopernicusData', '#biometricData', '#data', '#CopernicusData',

In [16]:
for term in lexdrafter:
    res = query_vectorstore(term['Term'])
    print(term['Term'])
    print([r['metadata']['definendum_label'] for r in res])
    print()


rules
['#nationalRules', '#nationalRules', '#bindingCorporateRules', '#generalBindingRules', '#updating-recording', '#TRACES', '#end-user', '#cross-borderCrime', '#alComma2', '#end-user']

generic ecodesign requirement
['#eco', '#eco40-60', '#equivalentModel', '#co-design', '#specificaGeneralePerLaProgettazioneEcocompatibile', '#specificaPerLaProgettazioneEcocompatibile', '#specificaParticolarePerLaProgettazioneEcocompatibile', '#progettazioneEcocompatibile', '#generalBindingRules', '#centraleTermoelettrica']

refrigerating appliance
['#refrigeratingAppliance', '#refrigeratingAppliance', '#householdRefrigeratingAppliance', '#refrigeratingApplianceWithADirectSalesFunction', '#refrigeratingApplianceWithADirectSalesFunction', '#refrigerator', '#foodProcessingRefrigeratingApplianceWithADirectSalesFunction', '#foodProcessingRefrigeratingAppliancesWithADirectSalesFunction', '#dedicatedRefrigeratingAppliance', '#dedicatedRefrigeratingAppliance']

abandoned land
['#abandonedLand', '#unusedLand

In [ ]:
retrieval_terms = ["Traffic Data", ""]



In [14]:
generation_terms = []

({'configurable': {'user_id': '1', 'thread_id': '1', 'model': 'gpt-4o-mini'}},)

## Perform Evaluation

In [16]:
from LegalDefAgent.src.agent_eval import definitions_agent_eval
import uuid


definendum = "dog"
model = "gpt-4o-mini"

inputs = {"messages": [("user", f"what's the definition of {definendum}?")]}
configurable = {"configurable": {"user_id": "1", "thread_id": uuid.uuid4().hex, "model": model}}

async for msg in definitions_agent_eval.astream(inputs, configurable, stream_mode="values"):
    # stream_mode="updates" yields dictionaries with output keyed by node name
    chat = msg


In [17]:
chat['messages'][-2].content

"{'relevant_definitions': [2772, 5008, 8263]}"

## Calculate Metrics

In [ ]:
!pip install -q evaluate

In [ ]:
import json

model = ""

# Load the JSON data
with open(f'../evaluation/{model}_generation.json', 'r') as json_file:
    data = json.load(json_file)

In [ ]:
# Initialize lists to store predictions and references
predictions = []
references = []

# Iterate through each item in the JSON data
for item in data:
  celex_id = item["CELEX_ID"]
  if item["existing_record"] == ["NEW TERM"]:
    original_text = item['original_definition']
    term = item['term']
    new_term = f"'{term}'"
    original_text = original_text.replace(term, new_term)
    generated_text = item['generated_definition']

    if ((generated_text != "") and (generated_text != "NO JSON AS AN OUTPUT OBTAINED")):
      # Add the generated text to predictions list
      predictions.append(generated_text)

      # Add the original text to references list
      references_item = [original_text]  # References for this item
      references.append(references_item)


---

### BLEU

In [19]:
import evaluate
bleu = evaluate.load("bleu")


In [ ]:
# Compute the BLEU score
bleu4_results = bleu.compute(predictions=predictions, references=references, max_order=4)
print(f"BLEU-4: {bleu4_results['bleu']}")

In [ ]:
# Compute the BLEU score
bleu3_results = bleu.compute(predictions=predictions, references=references, max_order=3)
print(f"BLEU-3: {bleu3_results['bleu']}")

In [ ]:
# Compute the BLEU score
blue2_results = bleu.compute(predictions=predictions, references=references, max_order=2)
print(f"BLEU-2: {blue2_results['bleu']}")

In [ ]:
# Compute the BLEU score
blue1_results = bleu.compute(predictions=predictions, references=references, max_order=1)
print(f"BLEU-1: {blue1_results['bleu']}")

---

### BertScore

In [ ]:
import evaluate
bertscore = evaluate.load("bertscore")

In [ ]:
# bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="nlpaueb/legal-bert-base-uncased")

In [ ]:
# Extract the F1 scores
f1_scores = bertscore_results['f1']

# Calculate the mean of F1 scores
mean_f1 = sum(f1_scores) / len(f1_scores)

print(f"BERTScore_F1: {bertscore_results['f1']}")
print("--------")
print(f"Mean BERTScore F1: {mean_f1}")

In [ ]:
# Extract the precision scores
precision_scores = bertscore_results['precision']

# Calculate the mean of precision scores
mean_precision = sum(precision_scores) / len(precision_scores)

print(f"BERTScore_precision: {bertscore_results['precision']}")
print("--------")
print(f"Mean BERTScore precision: {mean_precision}")

In [ ]:
# Extract the recall scores
recall_scores = bertscore_results['recall']

# Calculate the mean of recall scores
mean_recall = sum(recall_scores) / len(recall_scores)

print(f"BERTScore_recall: {bertscore_results['recall']}")
print("--------")
print(f"Mean BERTScore recall: {mean_recall}")

---

### BLEURT
>BLEURT is an evaluation metric for Natural Language Generation. It takes a pair of sentences as input, a reference and a candidate, and it returns a score that indicates to what extent the candidate is fluent and conveys the meaning of the reference.